In [89]:

# requirements: pdfplumber, pandas, tabula-py , numpy
import re, pdfplumber, pandas as pd, numpy as np
import mysql.connector 
from sqlalchemy import create_engine
from pathlib import Path
from mysql.connector import errorcode



In [90]:
PDFS = [
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_ICC.pdf",
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_ITI.pdf",
    "pdfs/PA_OTOÑO_2025_SEMESTRAL_LCC.pdf",
]

In [91]:
def clean_header(cols):
    return [re.sub(r"\s+", " ", c).strip().lower() for c in cols]

def extract_tables_pdfplumber(pdf_path):
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for table in page.extract_tables():
                if not table or len(table) < 2: 
                    continue
                header = clean_header(table[0])
                # heurística: columnas esperadas
                if {"nrc","clave","materia","días","hora","profesor","salón"}.issubset(set(header)) or \
                   {"nrc","clave","materia","dias","hora","profesor","salon"}.issubset(set(header)):
                    for r in table[1:]:
                        if r and any(x for x in r):
                            rows.append(dict(zip(header, r)))
    return pd.DataFrame(rows) if rows else pd.DataFrame()

def extract_all():
    frames = []
    for p in PDFS:
        if Path(p).exists():
            df = extract_tables_pdfplumber(p)
            if not df.empty:
                df["origen_pdf"] = Path(p).name
                frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

raw = extract_all()


In [92]:
# Normalización de encabezados frecuentes
raw = raw.rename(columns={
    "dias": "días", "salon": "salón"
})

# Limpieza de profesor
def normalizar_profesor(x: str):
    if not isinstance(x, str):
        return None
    x = re.sub(r"\s+", " ", x).strip()
    x = x.replace(" - ", " ")
    return x.title()

raw["profesor"] = raw["profesor"].apply(normalizar_profesor)

# Pasar todo a minúsculas y eliminar espacios
raw.columns = [c.strip().lower() for c in raw.columns]

# Asegurar que exista la columna 'hora'
if "horario" in raw.columns and "hora" not in raw.columns:
    raw.rename(columns={"horario": "hora"}, inplace=True)
elif "hora " in raw.columns:
    raw.rename(columns={"hora ": "hora"}, inplace=True)
elif "hora\n" in raw.columns:
    raw.rename(columns={"hora\n": "hora"}, inplace=True)
elif "h" in raw.columns:  # casos raros de extracción truncada
    raw.rename(columns={"h": "hora"}, inplace=True)

# Si sigue sin existir, crear una columna vacía para evitar errores posteriores
if "hora" not in raw.columns:
    raw["hora"] = None

# ---------------------------------------------------------
# Función para parsear horas de forma robusta
# ---------------------------------------------------------
def parse_hora(rango):
    """Parses hour ranges like '0700-0859', '07:00-08:59', '7:00 - 8:59'."""
    if not isinstance(rango, str):
        return pd.Series([None, None, None])
    
    s = rango.strip()
    s = re.sub(r"\s+", "", s)
    if s.lower() in ["nan", "none", ""]:
        return pd.Series([None, None, None])

    patron = r"(\d{1,2}):?(\d{2})-(\d{1,2}):?(\d{2})"
    m = re.match(patron, s)
    if not m:
        return pd.Series([None, None, None])

    h1, m1, h2, m2 = map(int, m.groups())
    start = pd.to_datetime(f"{h1:02d}:{m1:02d}", format="%H:%M", errors="coerce")
    end   = pd.to_datetime(f"{h2:02d}:{m2:02d}", format="%H:%M", errors="coerce")

    if pd.isna(start) or pd.isna(end):
        return pd.Series([None, None, None])
    duracion = int((end - start).total_seconds() / 60)
    if duracion <= 0:
        return pd.Series([None, None, None])

    return pd.Series([start.time(), end.time(), duracion])

# ---------------------------------------------------------
# Aplicar la función y crear las tres columnas
# ---------------------------------------------------------
raw[["hora_inicio", "hora_fin", "duracion_min"]] = raw["hora"].apply(parse_hora)


In [93]:
print(raw.columns.tolist())


['nrc', 'clave', 'materia', 'secc', 'días', 'hora', 'profesor', 'salón', '', 'origen_pdf', 'hora_inicio', 'hora_fin', 'duracion_min']


In [94]:
DIA_MAP = {"L":"Lunes","A":"Martes","M":"Miercoles",
           "J":"Jueves","V":"Viernes","S":"Sábado"}

# Expandir por múltiples días en una sola fila (si aplica)
def explotar_por_dia(df):
    out = []
    for _, row in df.iterrows():
        dias = str(row["días"]).replace(" ", "")
        if "," in dias:
            tokens = dias.split(",")
        else:
            tokens = list(dias)  # "AJL" -> ["A","J","L"]
        for d in tokens:
            r = row.copy()
            r["dia_codigo"] = d
            r["dia_semana"] = DIA_MAP.get(d, d)
            out.append(r)
    return pd.DataFrame(out)

curated = explotar_por_dia(raw)


In [95]:
# Salón -> edificio/aula: "1CCO4/203" -> edificio=1CCO4, aula=203
def split_salon(s):
    if not isinstance(s, str): return pd.Series([None, None, None])
    s = s.strip()
    m = re.match(r"([^/]+)/?(\w+)?", s)
    if not m: return pd.Series([s, None, s])
    edificio, aula = m.group(1), m.group(2)
    return pd.Series([edificio, aula, s])

curated[["edificio","aula","codigo_salon"]] = curated["salón"].apply(split_salon)

# Dimensiones (surrogate keys)
def build_dim(df, col_key, cols_keep, start_id=1, name_id="id"):
    d = df[cols_keep].drop_duplicates().reset_index(drop=True)
    d.insert(0, name_id, range(start_id, start_id+len(d)))
    return d

dim_docente = build_dim(curated, "profesor", ["profesor"], name_id="id_docente")
dim_materia = build_dim(curated, "materia", ["clave","materia"], name_id="id_materia")
dim_espacio = build_dim(curated, "codigo_salon", ["edificio","aula","codigo_salon"], name_id="id_espacio")

# dim_tiempo por fila (día + rango)
dim_tiempo = curated[["dia_codigo","dia_semana","hora_inicio","hora_fin"]].drop_duplicates().reset_index(drop=True)
dim_tiempo.insert(0, "id_tiempo", range(1, len(dim_tiempo)+1))

# Hechos (join a dimensiones)
def map_id(df, dim, key_cols_df, key_cols_dim, id_col):
    if isinstance(key_cols_df, str):
        key_cols_df = [key_cols_df]
    if isinstance(key_cols_dim, str):
        key_cols_dim = [key_cols_dim]

    df["_key_"] = df[key_cols_df].astype(str).agg("|".join, axis=1)
    dim["_key_"] = dim[key_cols_dim].astype(str).agg("|".join, axis=1)

    merged = df.merge(dim[["_key_", id_col]], on="_key_", how="left", validate="m:1")
    result = merged[id_col].values

    df.drop(columns="_key_", inplace=True, errors="ignore")
    dim.drop(columns="_key_", inplace=True, errors="ignore")

    return result

hechos = curated.copy()
hechos["id_docente"] = map_id(hechos, dim_docente, "profesor", "profesor", "id_docente")
hechos["id_materia"] = map_id(hechos, dim_materia, ["clave","materia"], ["clave","nombre_materia" if "nombre_materia" in dim_materia.columns else "materia"], "id_materia")
hechos["id_espacio"] = map_id(hechos, dim_espacio, "codigo_salon", "codigo_salon", "id_espacio")
hechos = hechos.merge(dim_tiempo, on=["dia_codigo","dia_semana","hora_inicio","hora_fin"], how="left")

print("Filas sin id_materia:", hechos["id_materia"].isna().sum())

hechos_clase = hechos[[
    "id_docente"    ,"id_materia","id_espacio","id_tiempo",
    "nrc","clave","secc" if "secc" in hechos.columns else "sección" if "sección" in hechos.columns else "días",
    "duracion_min"
]].rename(columns=lambda c: {"días":"seccion"}.get(c, c))


Filas sin id_materia: 0


In [96]:
"""-------------------------------------------------------------
# Guardar los resultados intermedios en archivos CSV
#-------------------------------------------------------------
output_dir = Path("data_export")
output_dir.mkdir(exist_ok=True)

#print(">> Guardando archivos CSV intermedios en ./data_export/")

raw.to_csv(output_dir / "data_raw.csv", index=False, encoding="utf-8-sig")
curated.to_csv(output_dir / "data_curated.csv", index=False, encoding="utf-8-sig")
hechos_clase.to_csv(output_dir / "data_hechos.csv", index=False, encoding="utf-8-sig")
dim_docente.to_csv(output_dir / "dim_docente.csv", index=False, encoding="utf-8-sig")
dim_materia.to_csv(output_dir / "dim_materia.csv", index=False, encoding="utf-8-sig")
dim_espacio.to_csv(output_dir / "dim_espacio.csv", index=False, encoding="utf-8-sig")
dim_tiempo.to_csv(output_dir / "dim_tiempo.csv", index=False, encoding="utf-8-sig")

print(">> Archivos CSV guardados correctamente en la carpeta 'data_export'")
"""

'-------------------------------------------------------------\n# Guardar los resultados intermedios en archivos CSV\n#-------------------------------------------------------------\noutput_dir = Path("data_export")\noutput_dir.mkdir(exist_ok=True)\n\n#print(">> Guardando archivos CSV intermedios en ./data_export/")\n\nraw.to_csv(output_dir / "data_raw.csv", index=False, encoding="utf-8-sig")\ncurated.to_csv(output_dir / "data_curated.csv", index=False, encoding="utf-8-sig")\nhechos_clase.to_csv(output_dir / "data_hechos.csv", index=False, encoding="utf-8-sig")\ndim_docente.to_csv(output_dir / "dim_docente.csv", index=False, encoding="utf-8-sig")\ndim_materia.to_csv(output_dir / "dim_materia.csv", index=False, encoding="utf-8-sig")\ndim_espacio.to_csv(output_dir / "dim_espacio.csv", index=False, encoding="utf-8-sig")\ndim_tiempo.to_csv(output_dir / "dim_tiempo.csv", index=False, encoding="utf-8-sig")\n\nprint(">> Archivos CSV guardados correctamente en la carpeta \'data_export\'")\n'

In [97]:
print(hechos_clase.isna().sum())


id_docente      0
id_materia      0
id_espacio      0
id_tiempo       0
nrc             0
clave           0
secc            0
duracion_min    0
dtype: int64


In [98]:
print(raw.columns.tolist())


['nrc', 'clave', 'materia', 'secc', 'días', 'hora', 'profesor', 'salón', '', 'origen_pdf', 'hora_inicio', 'hora_fin', 'duracion_min']


In [99]:
# -------------------------------------------------------------
# Limpieza y validación robusta de horas antes de la carga
# -------------------------------------------------------------

# Normaliza tipos de hora
raw["hora_inicio"] = pd.to_datetime(raw["hora_inicio"], errors="coerce").dt.time
raw["hora_fin"] = pd.to_datetime(raw["hora_fin"], errors="coerce").dt.time

# Quita filas sin hora válida
raw = raw.dropna(subset=["hora_inicio", "hora_fin"]).reset_index(drop=True)

# Filtra filas donde hora_fin <= hora_inicio
def es_valida(row):
    try:
        return row["hora_fin"] > row["hora_inicio"]
    except Exception:
        return False

mask_validas = raw.apply(es_valida, axis=1)
raw = raw.loc[mask_validas].copy().reset_index(drop=True)

# Recalcula duración por consistencia
def calcular_duracion(row):
    try:
        start = pd.to_datetime(str(row["hora_inicio"]), format="%H:%M:%S")
        end = pd.to_datetime(str(row["hora_fin"]), format="%H:%M:%S")
        dur = int((end - start).total_seconds() / 60)
        return dur if dur > 0 else None
    except Exception:
        return None

raw["duracion_min"] = raw.apply(calcular_duracion, axis=1)
raw = raw.dropna(subset=["duracion_min"]).reset_index(drop=True)


In [84]:
print(raw.columns.tolist())


['duracion_min']


In [100]:
print(raw[["hora_inicio", "hora_fin", "duracion_min"]].info())
print(raw[["hora_inicio", "hora_fin", "duracion_min"]].head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   hora_inicio   0 non-null      datetime64[ns]
 1   hora_fin      0 non-null      datetime64[ns]
 2   duracion_min  0 non-null      float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 132.0 bytes
None
Empty DataFrame
Columns: [hora_inicio, hora_fin, duracion_min]
Index: []


In [102]:
# pip install mysql-connector-python

# ---------------------------------------------------------------------------
# 🔧 CONFIGURACIÓN GLOBAL
# ---------------------------------------------------------------------------

DB_CONFIG = {
    "user": "root",
    "password": "1Encripta5",  # ⚠️ cambia por tu contraseña real
    "host": "localhost",
    "database": "horariosCubo",
    "allow_local_infile": True
}

# ---------------------------------------------------------------------------
# 🔗 CONEXIÓN A MYSQL
# ---------------------------------------------------------------------------

try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("✅ Conexión a MySQL establecida correctamente.")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("❌ Error: usuario o contraseña incorrectos.")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("❌ Error: la base de datos no existe.")
    else:
        print(f"❌ Error al conectar a MySQL: {err}")
    raise SystemExit()

# ---------------------------------------------------------------------------
# 🧱 CREACIÓN DE TABLAS (modelo estrella)
# ---------------------------------------------------------------------------

ddl_statements = [
    """
    CREATE TABLE IF NOT EXISTS dim_docente (
        id_docente INT PRIMARY KEY,
        nombre_completo VARCHAR(200)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_materia (
        id_materia INT PRIMARY KEY,
        clave VARCHAR(50),
        nombre_materia VARCHAR(200)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_espacio (
        id_espacio INT PRIMARY KEY,
        edificio VARCHAR(50),
        aula VARCHAR(50),
        codigo_salon VARCHAR(100)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_tiempo (
        id_tiempo INT PRIMARY KEY,
        dia_codigo VARCHAR(10),
        dia_semana VARCHAR(20),
        hora_inicio TIME,
        hora_fin TIME
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS hechos_clase (
        id_hecho INT AUTO_INCREMENT PRIMARY KEY,
        id_docente INT,
        id_materia INT,
        id_espacio INT,
        id_tiempo INT,
        nrc VARCHAR(20),
        clave VARCHAR(50),
        seccion VARCHAR(50),
        duracion_min INT,
        FOREIGN KEY (id_docente) REFERENCES dim_docente(id_docente),
        FOREIGN KEY (id_materia) REFERENCES dim_materia(id_materia),
        FOREIGN KEY (id_espacio) REFERENCES dim_espacio(id_espacio),
        FOREIGN KEY (id_tiempo) REFERENCES dim_tiempo(id_tiempo)
    )
    """
]

print(">> Creando tablas si no existen...")
for ddl in ddl_statements:
    cursor.execute(ddl)
conn.commit()
print("✅ Tablas creadas o verificadas correctamente.")

# ---------------------------------------------------------------------------
# 🧹 Limpieza y normalización
# ---------------------------------------------------------------------------

# Renombrar columnas si existen
if "profesor" in dim_docente.columns:
    dim_docente.rename(columns={"profesor": "nombre_completo"}, inplace=True)
if "materia" in dim_materia.columns:
    dim_materia.rename(columns={"materia": "nombre_materia"}, inplace=True)

# Normalizar nombre de columna 'secc' -> 'seccion'
if "secc" in hechos_clase.columns:
    hechos_clase.rename(columns={"secc": "seccion"}, inplace=True)
elif "sección" in hechos_clase.columns:
    hechos_clase.rename(columns={"sección": "seccion"}, inplace=True)
elif "días" in hechos_clase.columns:
    hechos_clase.rename(columns={"días": "seccion"}, inplace=True)

# Reemplazar NaN y valores "nan" o "NaT" por None
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.replace({np.nan: None, "nan": None, "NaN": None, "NaT": None})
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).replace(
                {"nan": None, "None": None, "NaN": None, "NaT": None, "": None}
            )
    return df

for df_name, df in {
    "dim_docente": dim_docente,
    "dim_materia": dim_materia,
    "dim_espacio": dim_espacio,
    "dim_tiempo": dim_tiempo,
    "hechos_clase": hechos_clase,
}.items():
    locals()[df_name] = clean_dataframe(df)
    print(f"NaN limpiados en {df_name}")

# ---------------------------------------------------------------------------
# 🧹 Limpieza previa en base de datos
# ---------------------------------------------------------------------------

print(">> Limpiando tablas existentes...")
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

for table in ["hechos_clase", "dim_docente", "dim_materia", "dim_espacio", "dim_tiempo"]:
    cursor.execute(f"TRUNCATE TABLE {table}")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")
conn.commit()
print("✅ Tablas limpiadas correctamente (Foreign Keys reactivadas).")

# ---------------------------------------------------------------------------
# 🧠 Helper: inserción segura de DataFrames
# ---------------------------------------------------------------------------

def insert_dataframe(df: pd.DataFrame, table_name: str):
    if df.empty:
        print(f"(⚠️ {table_name} está vacío, no se inserta nada)")
        return
    cols = ", ".join(df.columns)
    placeholders = ", ".join(["%s"] * len(df.columns))
    sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    data = []
    for row in df.itertuples(index=False, name=None):
        clean_row = tuple(
            None if (isinstance(x, float) and pd.isna(x)) or str(x).lower() in ["nan", "nat", "none", ""] else x
            for x in row
        )
        data.append(clean_row)
    cursor.executemany(sql, data)
    conn.commit()
    print(f"✅ {len(df)} filas insertadas en {table_name}")

# ---------------------------------------------------------------------------
# 🚀 Inserción de datos (ajustada para mantener coherencia con el cubo)
# ---------------------------------------------------------------------------

# Convertir columnas de hora a formato HH:MM:SS
for col in ["hora_inicio", "hora_fin"]:
    if col in dim_tiempo.columns:
        def to_mysql_time(x):
            if pd.isna(x) or x in [None, "", "None", "nan", "NaT"]:
                return None
            # si es datetime.time
            if hasattr(x, "strftime"):
                return x.strftime("%H:%M:%S")
            # si es string, intenta parsear
            try:
                return pd.to_datetime(x, errors="coerce").strftime("%H:%M:%S")
            except Exception:
                return None
        dim_tiempo[col] = dim_tiempo[col].apply(to_mysql_time)

insert_dataframe(dim_docente, "dim_docente")
insert_dataframe(dim_materia, "dim_materia")
insert_dataframe(dim_espacio, "dim_espacio")
insert_dataframe(dim_tiempo, "dim_tiempo")

# Asegurar tipo entero antes de insertar
if "duracion_min" in hechos_clase.columns:
    hechos_clase["duracion_min"] = hechos_clase["duracion_min"].astype(int)

print("Columnas en hechos_clase:", hechos_clase.columns.tolist())
insert_dataframe(hechos_clase, "hechos_clase")

print("✅ Todos los datos cargados exitosamente en MySQL.")

# ---------------------------------------------------------------------------
# 🔚 Cierre
# ---------------------------------------------------------------------------
cursor.close()
conn.close()
print("🔚 Conexión a MySQL cerrada.")


✅ Conexión a MySQL establecida correctamente.
>> Creando tablas si no existen...
✅ Tablas creadas o verificadas correctamente.
NaN limpiados en dim_docente
NaN limpiados en dim_materia
NaN limpiados en dim_espacio
NaN limpiados en dim_tiempo
NaN limpiados en hechos_clase
>> Limpiando tablas existentes...
✅ Tablas limpiadas correctamente (Foreign Keys reactivadas).
✅ 105 filas insertadas en dim_docente
✅ 96 filas insertadas en dim_materia
✅ 42 filas insertadas en dim_espacio
✅ 47 filas insertadas en dim_tiempo
Columnas en hechos_clase: ['id_docente', 'id_materia', 'id_espacio', 'id_tiempo', 'nrc', 'clave', 'seccion', 'duracion_min']
✅ 1184 filas insertadas en hechos_clase
✅ Todos los datos cargados exitosamente en MySQL.
🔚 Conexión a MySQL cerrada.


In [114]:
# -------------------------------------------------------------
# 🔗 CONEXIÓN A MYSQL
# -------------------------------------------------------------
import datetime
DB_USER = "root"
DB_PASS = "1Encripta5"
DB_HOST = "localhost"
DB_NAME = "horariosCubo"

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
print("✅ Conectado correctamente a la base de datos MySQL.")

# -------------------------------------------------------------
# 📥 CARGA DE TABLAS DESDE MYSQL
# -------------------------------------------------------------
dim_docente = pd.read_sql("SELECT * FROM dim_docente", engine)
dim_materia = pd.read_sql("SELECT * FROM dim_materia", engine)
dim_espacio = pd.read_sql("SELECT * FROM dim_espacio", engine)
dim_tiempo  = pd.read_sql("SELECT * FROM dim_tiempo", engine)
hechos_clase = pd.read_sql("SELECT * FROM hechos_clase", engine)
print("✅ Tablas cargadas correctamente desde MySQL.")

def convertir_a_time(x):
    if x is None or pd.isna(x):
        return None
    elif isinstance(x, datetime.time):
        return x
    elif isinstance(x, datetime.timedelta):
        # Convierte timedelta a time
        total_seconds = int(x.total_seconds())
        horas = total_seconds // 3600
        minutos = (total_seconds % 3600) // 60
        segundos = total_seconds % 60
        return datetime.time(horas, minutos, segundos)
    elif isinstance(x, str):
        try:
            return pd.to_datetime(x, format="%H:%M:%S").time()
        except Exception:
            return None
    else:
        return None

for col in ["hora_inicio", "hora_fin"]:
    if col in dim_tiempo.columns:
        dim_tiempo[col] = dim_tiempo[col].apply(convertir_a_time)

print("✅ Conversión completada.")
print(dim_tiempo.dtypes)
print(dim_tiempo.head())
print("Ejemplo de tipo:", type(dim_tiempo.loc[0, "hora_inicio"]))

✅ Conectado correctamente a la base de datos MySQL.
✅ Tablas cargadas correctamente desde MySQL.
✅ Conversión completada.
id_tiempo       int64
dia_codigo     object
dia_semana     object
hora_inicio    object
hora_fin       object
dtype: object
   id_tiempo dia_codigo dia_semana hora_inicio  hora_fin
0          1          A     Martes    07:00:00  08:59:00
1          2          J     Jueves    07:00:00  08:59:00
2          3          A     Martes    09:00:00  10:59:00
3          4          J     Jueves    09:00:00  10:59:00
4          5          M  Miercoles    17:00:00  18:59:00
Ejemplo de tipo: <class 'datetime.time'>


In [120]:
# -------------------------------------------------------------
# 🧠 CLASE: CuboHorario
# -------------------------------------------------------------
class CuboHorario:
    def __init__(self, hechos, dim_docente, dim_materia, dim_espacio, dim_tiempo):
        self.hechos = hechos
        self.dim_docente = dim_docente
        self.dim_materia = dim_materia
        self.dim_espacio = dim_espacio
        self.dim_tiempo = dim_tiempo

        # Unir hechos con dimensiones
        self.cubo = (
            hechos
            .merge(dim_docente, on="id_docente", how="left")
            .merge(dim_materia, on="id_materia", how="left")
            .merge(dim_espacio, on="id_espacio", how="left")
            .merge(dim_tiempo, on="id_tiempo", how="left")
        )

        # Normalizar columnas duplicadas
        if "clave_x" in self.cubo.columns and "clave_y" in self.cubo.columns:
            self.cubo["clave"] = self.cubo["clave_y"].combine_first(self.cubo["clave_x"])
            self.cubo.drop(columns=["clave_x", "clave_y"], inplace=True)
        elif "clave_x" in self.cubo.columns:
            self.cubo.rename(columns={"clave_x": "clave"}, inplace=True)
        elif "clave_y" in self.cubo.columns:
            self.cubo.rename(columns={"clave_y": "clave"}, inplace=True)

        # Conversión de hora segura (asegura datetime.time)
        for col in ["hora_inicio", "hora_fin"]:
            if col in self.cubo.columns:
                def to_time_safe(x):
                    if pd.isna(x) or x in [None, "", "NaT", "None"]:
                        return None
                    if hasattr(x, "hour"):
                        return getattr(x, "time", lambda: x)()
                    try:
                        return pd.to_datetime(str(x), errors="coerce").time()
                    except Exception:
                        return None
                self.cubo[col] = self.cubo[col].apply(to_time_safe)

        print("🧩 Columnas finales en el cubo:")
        print(self.cubo.columns.tolist())

    # 1️⃣ Horario semanal de un docente
    def horario_docente(self, nombre_docente):
        df = self.cubo[self.cubo["nombre_completo"].str.contains(nombre_docente, case=False, na=False)]
        df = df.sort_values(["dia_semana", "hora_inicio"])
        if df.empty:
            return f"No se encontraron clases para el docente '{nombre_docente}'."
        return df[[
            "nombre_completo", "dia_semana", "hora_inicio", "hora_fin",
            "nombre_materia", "clave", "codigo_salon", "edificio", "aula"
        ]]

    # 2️⃣ Docentes que imparten una materia específica
    def docentes_por_materia(self, materia_o_clave):
        cols = self.cubo.columns
        condiciones = []
        if "nombre_materia" in cols:
            condiciones.append(self.cubo["nombre_materia"].str.contains(materia_o_clave, case=False, na=False))
        if "clave" in cols:
            condiciones.append(self.cubo["clave"].str.contains(materia_o_clave, case=False, na=False))
        if not condiciones:
            raise KeyError("No se encontró ninguna columna 'nombre_materia' o 'clave' en el cubo.")
        filtro = condiciones[0]
        for cond in condiciones[1:]:
            filtro = filtro | cond
        df = self.cubo[filtro]
        if df.empty:
            return f"No se encontraron docentes para la materia '{materia_o_clave}'."
        return (df[["clave", "nombre_materia", "nombre_completo"]]
                .drop_duplicates()
                .sort_values(["clave", "nombre_completo"]))

    # 3️⃣ Docentes en un edificio a una hora específica
    def docentes_en_edificio_hora(self, edificio, hora):
        try:
            hora_ref = pd.to_datetime(hora, format="%H:%M", errors="coerce").time()
        except Exception:
            raise ValueError("Formato de hora inválido. Usa HH:MM (ejemplo: '09:30').")

        df = self.cubo.dropna(subset=["hora_inicio", "hora_fin"]).copy()

        # Comparación segura
        mask = (
            df["edificio"].astype(str).str.contains(edificio, case=False, na=False)
        ) & (
            df["hora_inicio"].apply(lambda h: isinstance(h, datetime.time) and h <= hora_ref)
        ) & (
            df["hora_fin"].apply(lambda h: isinstance(h, datetime.time) and h >= hora_ref)
        )

        df = df.loc[mask]

        if df.empty:
            return f"No hay docentes en el edificio '{edificio}' a las {hora_ref}."

        return (
            df[["edificio", "aula", "nombre_completo", "nombre_materia", "hora_inicio", "hora_fin"]]
            .sort_values(["aula", "nombre_completo"])
            .reset_index(drop=True)
        )

    # 4️⃣ Clases por docente y día (ordenadas de lunes a sábado)
    def clases_por_docente_y_dia(self):
        tabla = pd.pivot_table(
            self.cubo,
            values="nrc",
            index="nombre_completo",
            columns="dia_semana",
            aggfunc="count",
            fill_value=0
        )

        # Orden correcto de días
        orden_dias = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]

        # Reordenar columnas según el orden de la lista (solo las que existan)
        columnas_presentes = [dia for dia in orden_dias if dia in tabla.columns]
        tabla = tabla[columnas_presentes]

        # Agregar total semanal
        tabla["Total"] = tabla.sum(axis=1)

        return tabla

    # 5️⃣ Duración promedio por edificio
    def promedio_duracion_por_edificio(self):
        df = self.cubo.groupby("edificio")["duracion_min"].mean().reset_index()
        df.rename(columns={"duracion_min": "duracion_promedio_min"}, inplace=True)
        return df.sort_values("duracion_promedio_min", ascending=False)


# -------------------------------------------------------------
# 🚀 CREAR EL CUBO Y PROBAR CONSULTAS
# -------------------------------------------------------------
cubo = CuboHorario(hechos_clase, dim_docente, dim_materia, dim_espacio, dim_tiempo)


🧩 Columnas finales en el cubo:
['id_hecho', 'id_docente', 'id_materia', 'id_espacio', 'id_tiempo', 'nrc', 'seccion', 'duracion_min', 'nombre_completo', 'nombre_materia', 'edificio', 'aula', 'codigo_salon', 'dia_codigo', 'dia_semana', 'hora_inicio', 'hora_fin', 'clave']


In [123]:

print("\n📅 Horario de 'Mariano':")
print(cubo.horario_docente("Mariano"))

print("\n📘 Docentes que imparten 'Estructuras de Datos':")
print(cubo.docentes_por_materia("Estructuras de Datos"))

print("\n🏫 Docentes en edificio '1CCO4' a las 09:30:")
print(cubo.docentes_en_edificio_hora("1CCO4", "09:30"))
"""
print("\n📊 Clases por docente y día:")
print(cubo.clases_por_docente_y_dia())

print("\n⏱️ Duración promedio por edificio:")
print(cubo.promedio_duracion_por_edificio())
"""


📅 Horario de 'Mariano':
           nombre_completo dia_semana hora_inicio  hora_fin  \
362   Larios Gomez Mariano     Jueves    11:00:00  12:59:00   
404   Larios Gomez Mariano     Jueves    15:00:00  16:59:00   
606   Larios Gomez Mariano      Lunes    11:00:00  11:59:00   
363   Larios Gomez Mariano      Lunes    12:00:00  12:59:00   
1098  Larios Gomez Mariano      Lunes    15:00:00  15:59:00   
405   Larios Gomez Mariano      Lunes    16:00:00  16:59:00   
361   Larios Gomez Mariano     Martes    11:00:00  12:59:00   
403   Larios Gomez Mariano     Martes    15:00:00  16:59:00   
607   Larios Gomez Mariano  Miercoles    11:00:00  12:59:00   
1099  Larios Gomez Mariano  Miercoles    15:00:00  16:59:00   
608   Larios Gomez Mariano    Viernes    11:00:00  12:59:00   
1100  Larios Gomez Mariano    Viernes    15:00:00  16:59:00   

                      nombre_materia     clave codigo_salon edificio aula  
362                  Modelo de Redes  ICCS 259    1CCO1/004    1CCO1  004  
404

'\nprint("\n📊 Clases por docente y día:")\nprint(cubo.clases_por_docente_y_dia())\n\nprint("\n⏱️ Duración promedio por edificio:")\nprint(cubo.promedio_duracion_por_edificio())\n'